This is for testing SEI growth. We have noticed that:

In the case of "electron-migration limited" mechanism:
 1. The SEI fluctuates (decreases) for default parameters (i.e. without using any parameter sets) and the "Chen2020", "Ai2020", "Marquis2019", and "Mohtat2020" parameter sets.
 2. Get error when using "OKane2022" parameter set.

In the case of "ec reaction limited" mechanism:
The initial inner and outer SEI thicknesses must be greater than 3e-12.


In [97]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
param = pybamm.ParameterValues("OKane2022")


# param['Initial inner SEI thickness [m]'] = 2.0e-12
# param['Initial outer SEI thickness [m]'] = 2.0e-12


cycle_no = 2
exp_CCCV = pybamm.Experiment(
    [("Charge at C/10 until 4.2 V",
        "Rest for 10 hours",
      "Discharge at C/10 until 2.5 V",
      )] * cycle_no
)

# Options of SEI Mechanism:
SEI_Mech = ["reaction limited", "solvent-diffusion limited",
            "interstitial-diffusion limited", "ec reaction limited", "electron-migration limited"]
index = 4
SPM_model = pybamm.lithium_ion.SPM(options={
    "SEI": SEI_Mech[index],
    "SEI porosity change": "true", })

sims_SPM = pybamm.Simulation(SPM_model, parameter_values=param,
                             experiment=exp_CCCV)
sol = sims_SPM.solve(initial_soc=0)

plt.plot(sol["Time [h]"].entries,
         sol['X-averaged total SEI thickness [m]'].entries)

The right-hand side routine failed at the first call.
The right-hand side routine failed at the first call.
The right-hand side routine failed at the first call.
The right-hand side routine failed at the first call.
The right-hand side routine failed at the first call.
The right-hand side routine failed at the first call.
2023-11-01 13:44:51.086 - [ERROR] callbacks.on_experiment_error(224): Simulation error: Maximum number of decreased steps occurred at t=0.0 (final SolverError: 'Error in Function::call for 'F' [CvodesInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'F' [CvodesInterface] at .../casadi/core/function.cpp:330:
.../casadi/interfaces/sundials/cvodes_interface.cpp:392: CVode returned "CV_FIRST_RHSFUNC_ERR". Consult CVODES documentation.'). For a full solution try reducing dt_max (currently, dt_max=600) and/or reducing the size of the time steps or period of the experiment. Set `return_solution_if_failed_early=True` to return the solution object up

SolverError: Maximum number of decreased steps occurred at t=0.0 (final SolverError: 'Error in Function::call for 'F' [CvodesInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'F' [CvodesInterface] at .../casadi/core/function.cpp:330:
.../casadi/interfaces/sundials/cvodes_interface.cpp:392: CVode returned "CV_FIRST_RHSFUNC_ERR". Consult CVODES documentation.'). For a full solution try reducing dt_max (currently, dt_max=600) and/or reducing the size of the time steps or period of the experiment. Set `return_solution_if_failed_early=True` to return the solution object up to the point where failure occured.